Add Packages

In [ ]:
#r "nuget:Microsoft.ML, *-*"
#r "nuget:Microsoft.ML.Recommender, *-*"
#r "nuget:Microsoft.Data.Analysis, *-*"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab, *-*"

Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22313.1 Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.22362.4 Microsoft.ML, 2.0.0-preview.22313.1 Microsoft.ML.Recommender, 0.20.0-preview.22313.1

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `Microsoft.DotNet.Interactive.ExtensionLab.dll`

Create strongly-typed dataframes using #!linqify . 
 The #!linqify magic command replaces a Microsoft.Data.Analysis.DataFrame variable with a generated, strongly-typed data frame, allowing the use of LINQ operations over the contained data.

ExplainCode 
 This extension generates Sequence diagrams from csharp code using Mermaid kernel.

Inspect code compilation details using the #!inspect magic command. 
 The #!inspect magic command allows you to see the C# decompilation, IL, and JIT Asm for the code in a C# cell.

Use the #!record magic command to keep a transcript of the code you run. 
 Once you enable transcripts using #!record , each code submission (including re-running cells) is recorded in the specified file. The JSON format used is the same format recognized by the .NET Interactive stdio and http APIs and can be used to replay an interactive session via automation.

Query SQLite databases. 
 This extension adds support for connecting to SQLite databases using the #!connect sqlite magic command. For more information, run a cell using the #!sql magic command.

In [ ]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;
using Microsoft.ML.Trainers.Recommender;

using System.IO;
using System.Text;

Data Models

In [ ]:
public class MovieRating
{
    [LoadColumn(1)]
    public int MovieId { get; set; }

    [LoadColumn(2)]
    public int UserId { get; set; }

    [LoadColumn(3)]
    public float Label { get; set; }
}

public class MovieRatingPrediction
{
    [ColumnName("Label")]
    public float Label;

    [ColumnName("Score")]
    public float Score;
}

Load And Split Data

In [ ]:
var mlContext = new MLContext();
var data = mlContext.Data.LoadFromTextFile<MovieRating>("./Data/netflix_subset.csv", hasHeader: true, separatorChar: ',');

// Split data into testing and training data
var dataSplit = mlContext.Data.TrainTestSplit(data, testFraction: 0.3);

Train The Model

In [ ]:
var model = mlContext.Recommendation().Trainers.MatrixFactorization(
    labelColumnName: "Label",
    matrixColumnIndexColumnName: "UserIdEncoded",
    matrixRowIndexColumnName: "MovieIdEncoded",
    approximationRank: 100,
    learningRate: 0.1,
    numberOfIterations: 50
);

var pipeline = mlContext.Transforms.Conversion.MapValueToKey(inputColumnName: "UserId", outputColumnName: "UserIdEncoded")
    .Append(mlContext.Transforms.Conversion.MapValueToKey(inputColumnName: "MovieId", outputColumnName: "MovieIdEncoded"))
    .Append(model);

var trainedModel = pipeline.Fit(dataSplit.TrainSet);

Evaluate The Model

In [ ]:
var testSetTransform = trainedModel.Transform(dataSplit.TestSet);
var metrics = mlContext.Recommendation().Evaluate(testSetTransform, labelColumnName: "Label", scoreColumnName: "Score");

In [ ]:
metrics

MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
0.9887091717088996,1.5879782082553315,1.2601500737036568,1.5879782080604088,-0.3478016509832822


Save The Model

In [ ]:
mlContext.Model.Save(trainedModel, dataSplit.TrainSet.Schema, "./RecommendationSystem.mdl");

Load The Model And Predict

In [ ]:
var newSample = new MovieRating
{
    UserId = 1488844,
    MovieId = 1
};

using (var stream = new FileStream("./RecommendationSystem.mdl", FileMode.Open, FileAccess.Read, FileShare.Read))
{
    var loadedModel = mlContext.Model.Load(stream, out _);
    var predictionEngine = mlContext.Model.CreatePredictionEngine<MovieRating, MovieRatingPrediction>(loadedModel);

    var prediction = predictionEngine.Predict(newSample);

    Console.WriteLine($"Prediction : {prediction.Score}");
}

Prediction : 3.0785866
